In [1]:
import pandas as pd
import numpy as np
import re
import spacy

In [2]:
pd.set_option('display.max_colwidth', 1000)

In [3]:
all_amz_go = pd.read_csv('F:\\mailae_g\\ALL_MAG_2021+2022.csv', sep=';' )
all_cus_go = pd.read_csv('F:\\mailae_g\\ALL_PL_2022.csv', sep=';')

In [4]:
all_amz_go.sample(5)

,Ticket Id,Subject,Description,Source,Requester Email,Created Time,Agent interactions,Customer interactions,Tags
11030,359147,Enquiry from Amazon customer Catherine(Order: 306-8934362-3213108),"Hallo, ich habe den Artikel zwei Wochen vorher verschickt. Bitte teilen Sie mir den Status der Rücksendung mit und veranlassen Sie eine Rückerstattung. Vielen Dank, Yajiao Chen hello, I have sent the item two weeks before. Please let me know the status of the return and issue a refund. Many thanks, Yajiao Chen",Email,freshdeskuser_24945807@channelreply.email,10 10 2022 18:32,4,3,"cr,rynek_de,framesfactory,framesfactory_de,FBM"
24585,197383,Richiesta da parte di marco cliente Amazon(Ordine: 406-9128625-3257166),Salve volevo sapere se la spedizione di 24€ è compreso nel rimborso Attached file: IMG-20220430-WA0008.jpg,Email,freshdeskuser_22261899@channelreply.email,30 04 2022 19:09,3,2,"cr,turborury,turborury_it,rynek_it"
7132,371831,Demande de renseignements concernant la livraison dune commande de la part du client Amazon pignon(Commande : 402-9198943-9765916),"Bonjour, Nous avons été contactés par un client concernant une commande passée auprès de votre boutique. Voici l&#39;information fournie par le client: Numéro de commande: 402-9198943-9765916 Article: Raison: Où est ma commande? Détails: L’article n’a pas été livré à la date prévue. client a vérifier par tt il a rien récu Merci de contacter le client afin de l’informer sur le statut de la livraison. Nous vous remercions de répondre directement à ce client en répondant à cet e-mail ou en vous rendant dans votre compte vendeur: [link removed] Nous espérons que vous pourrez régler cette question avec ce client et vous remercions d&#39;avance pour l&#39;attention que vous porterez à cette requête. Cordialement, le Service Client Amazon.fr http://www.amazon.fr",Email,freshdeskuser_25153039@channelreply.email,24 10 2022 10:47,13,6,"cr,rynek_fr,morelenet,morelenet_fr,FBM"
9593,365985,Demande de facturation de la part du client Amazon Marketplace eon(Commande : 403-5030045-9293137),bonjour merci de me faire parvenir la facture de ce produit - est ce possible de la mettre au nom de EI LESOURD Jérome le grand pommauger 53600 EVRON,Email,freshdeskuser_25062110@channelreply.email,18 10 2022 06:37,2,2,"cr,rynek_fr,morelenet,morelenet_fr,FBM"
13737,339331,Richiesta da parte di Fernanda cliente Amazon(Ordine: 171-3725891-0245917),Non è arrivato Chiedo rimborso,Email,freshdeskuser_24604835@channelreply.email,17 09 2022 23:01,9,6,"cr,rynek_it,sfd,sfd_it,FBM"


In [5]:
all_cus_go.sample(5)

,Ticket Id,Subject,Description,Source,Requester Email,Created Time,Agent interactions,Customer interactions,Tags
24207,341090,[DE] 23-09099-92411,"Dzień dobry, klient zamowieł zły rozmiar dlatego odesłała zamowienie na swoj koszt - DHL 358947885338 Pozdrawiam Rita",Outbound Email,kotarbauzapytania@gmail.com,20 09 2022 11:43,2,2,"Partner_wysłane,rynek_de,Przypomnienie1,Przypomnienie2,Przypomnienie3,Pending6,Pending8,Nieodpowiedziane"
40342,315219,[FR] shipping 408-0354316-0483536,"Dzień dobry, Klient kontaktuje się z nami, aby wskazać, że życzy sobie dostawy w piątek lub sobotę i że dostawa ma nastąpić w domu pod następującym adresem: 7 square du Velay 75013 Paryż numer telefonu 0612671527 Z wyrazami szacunku, Julita",Outbound Email,amazon@fabrykaram.pl,26 08 2022 07:39,5,3,"Partner_wysłane,rynek_fr,Przypomnienie"
11263,3021269,303-3098150-5037110,"Witam, Prosimy o napisanie wiadomości do klienta o braku obuwia. Pozdrawiamy Matt_Shoes Zespół Ulubioneobuwie",Email,matobuwie@wp.pl,29 11 2022 09:16,2,1,"Partner_email,Przychodzące_email,rynek_de"
3978,3040232,[UK] NIEDOSTARCZONA 205-7422336-4997908,"Dzień dobry, termin dostawy minął 6.12, a w śledzeniu paczka od 13.12 jest gotowa do dostarczenia. Klient prosi o weryfikację, co się dzieje z paczką. Pozdrawiam Wiktoria",Outbound Email,gonito@fkwt.pl,17 12 2022 11:42,5,4,"Partner_wysłane,rynek_uk"
26541,358999,de 303-2232113-5274719 anulowanie,Klient prosi o anulowanie zamówienia,Outbound Email,sklep@sfd.pl,10 10 2022 15:46,5,4,"rynek_de,Anulacja,Pilne"


In [6]:
all_amz_go.reset_index(drop=True, inplace=True)
all_cus_go.reset_index(drop=True, inplace=True)

In [7]:
# Funkcja do dzielenia komórek na listę, pierwszy argument tekst, drugi argument po czym tekst ma być dzielony 

def split_tags(tags: str, split_char: str) -> list:
    try:
        tags = tags.split(f'{split_char}')
        return tags 
    except Exception:
        return (list[tags])
    

In [8]:
# funkcja do wyciągania z listy, jakiego rynku dotyczą wiadomości

def get_market(lst: list) -> str:
    for i in lst:
        if "rynek" in i:
            return i

In [9]:
# funkcja do wyciągania numerów zamówień z tekstu i tytułu maila. Przyjmuje dwa różne ciągi tekstowe, sprawdza czy jest w tytule jest regex, jeśli nie to sprawdza czy jest w opisie, jeśli nie to zwraca pustą listę

se = re.compile(r"\d{3}-\d{7}-\d{7}")

def order_number(subject: str, description: str) -> list:
    subject = str(subject)
    description = str(description)
    if se.findall(subject) == []:
        return se.findall(description)
    return se.findall(subject)


In [10]:
# funkcja do tworzenia listy unikalnych wartości z kolumn. Dajemy wartość z kolumny i listę. Sprawdza czy wartość już znajduje się na liście, jeśli nie to ją dodaje

def all_order(order: list, new_list: list) -> list:
    try:
        if order[0] in new_list:
            return new_list
        else:
            new_list.append(order)
            return new_list
    except Exception:
        i =1

In [11]:
# druga funkcja do czyszczenia tytułów w celu zebrania etykiet do danych

on = re.compile(r"\d{3}-\d{7}-\d{7}")
cc = re.compile(r"w{2}")
cc2 = re.compile(r'\[\w{2}\]')

def clear_title(title_list: list) -> list:
    for el in title_list:
        print(el)
    list_to_remove = []
    for i in range(len(title_list)):
        if re.match(on, title_list[i]):
            title_list.remove(i)
        elif re.match(cc, title_list[i]):
            title_list.remove(i)
        elif re.match(cc2, title_list[i]):
            title_list.remove(i)
    return title_list

In [12]:
# trzecia funkcja do czyszczenia tytułów w celu zebrania etykiet do danych

def num_there(s):
    return any(i.isdigit() for i in s)

def find_words_with_more_than_four_characters(words_with_tags):
  words = []
  for i in words_with_tags:
    if num_there(i):
      continue
    w = re.sub('[^A-Za-zĄĆĘŁŃÓŚŹŻąćęłńóśźż]+', ' ', i)
    w = w.strip(" ")
    if len(w) >= 4:
      words.append(w)

  return words

In [13]:
# czwarta funkcja do czyszczenia tytułów w celu zebrania etykiet do danych

def find_words_with_more_than_four_characters2(subject: str) -> list:
  words = []
  new_subject = ''
  for l in subject:
    l = re.sub('[^A-Za-zĄĆĘŁŃÓŚŹŻąćęłńóśźż]+', ' ', l)
    new_subject += l
  words_temp = new_subject.split(' ')
  for i in words_temp:
    if len(i) >= 4:
      words.append(i)

  return words

In [14]:
# Funkcja do zliczania elementów w liscie, zwraca listę unikatowych wartości i listę unikatowych wartości z informacją ile razy występowały

from collections import Counter

def count_occurrences(input_list):
    unique_values = set(input_list)
    occurrence_count = Counter(input_list)

    return list(unique_values), occurrence_count

In [15]:
# Funkcja sprawdza czy peirwszy elemnt tupli jest kluczem do słownika, jeśli tak to dodaje 0 elemnt do listy i zwraca set, który zawiera unikatowe wartości dla danego klucza

def find_all_values_to_dictonary (key: str ,list_of_tuple: list) -> set:
    di = []
    for i in list_of_tuple:
        if i[1] == key:
            di.append(i[0])
    return set(di)


In [16]:
# Zmiana tytułów na lowercase

all_amz_go['Subject'] = [str(row).lower() for row in all_amz_go['Subject']]
all_cus_go['Subject'] = [str(row).lower() for row in all_cus_go['Subject']]

In [17]:
# przekształcenie stringów w komórkach kolumny tags na listę, dzielone po przecinku 

all_amz_go['Tags'] = [split_tags(row, ',')for row in all_amz_go['Tags']]
all_cus_go['Tags'] = [split_tags(row, ',')for row in all_cus_go['Tags']]

In [18]:
# utworzenie kolumny, w której jest napisane jakiego rynku dotyczy wiadomość, na podstawie kolumny tags

all_amz_go['rynek'] = [get_market(row) for row in all_amz_go['Tags']]
all_cus_go['rynek'] = [get_market(row) for row in all_cus_go['Tags']]

In [19]:
# usunięcie wierszy, które w kolumnie 'rynek' nie maję 'rynek_de'

# all_amz_go.drop(all_amz_go[all_amz_go['rynek'] != 'rynek_de'].index,  inplace=True)
# all_cus_go.drop(all_cus_go[all_cus_go['rynek'] != 'rynek_de'].index,  inplace=True)

In [20]:
# wyciągnięcie numeru zamówień z kolumn subject i description

all_amz_go['Order_number'] = [order_number(row1, row2) for row1, row2 in zip(all_amz_go['Subject'], all_amz_go['Description'])]
all_cus_go['Order_number'] = [order_number(row1, row2) for row1, row2 in zip(all_cus_go['Subject'], all_cus_go['Description'])]

In [21]:
# resetowanie indexów

all_amz_go.reset_index(drop=True, inplace=True)
all_cus_go.reset_index(drop=True, inplace=True)

In [22]:
# zmiana obiektów z kolummny 'Created Time' na daty

from datetime import datetime

format_date = "%d %m %Y %H:%M"

all_amz_go['Created Time'] = [datetime.strptime(row, format_date) for row in all_amz_go['Created Time']] 
all_cus_go['Created Time'] = [datetime.strptime(row, format_date) for row in all_cus_go['Created Time']] 

In [23]:
# wykorzystanie funkcji do splitowania aby utworzyć kolumnę 

all_amz_go['title_list'] = [split_tags(row, ' ') for row in all_amz_go['Subject']]
all_cus_go['title_list'] = [split_tags(row, ' ') for row in all_cus_go['Subject']]

In [24]:
# sprawdzenie czy funcja split_tags zadzaiała poprawnie

all_cus_go['title_list'][1]

['[de]', '302-0014409-7188317', 'niekompletne']

W tym momencie postanowieł sprawdzić, czy zamówienia, z jednej i drugiej tabeli się pokrywają w jakimś najmniejszym stopniu

In [25]:
# za pomocą wcześniej stworzonej funkcji tworzę listę unikatowych numerów ze wszystkich wierszy jednego data set

all_amz_go_list = []
[all_order(row, all_amz_go_list) for row in all_amz_go['Order_number']]
all_amz_go_list = [item for sublist in all_amz_go_list for item in sublist]

In [26]:
# za pomocą wcześniej stworzonej funkcji tworzę listę unikatowych numerów ze wszystkich wierszy drugiego data set

all_cus_go_list = []
[all_order(row, all_cus_go_list) for row in all_cus_go['Order_number']]
all_cus_go_list = [item for sublist in all_cus_go_list for item in sublist]

In [27]:
# zamieniam stworzyonych list na set

mcg_order_set = set(all_amz_go_list)
mag_rynek_de_order_set = set(all_cus_go_list)

In [28]:
# dodanie kontrolnie jednej takiej samej wartości do obu setów

mcg_order_set.add('123')
mag_rynek_de_order_set.add('123')

In [29]:
# porównanie dwóch

y = mcg_order_set.intersection(mag_rynek_de_order_set)

In [30]:
# Wyświetlenie ile mamy takich samych numerów zamówień w jednym i drugim secie

len(y)

11037

Powrót do preprocesingu danych

In [31]:
# weryfickaja działania funkcji do czyszczenia tytułów maili

print(find_words_with_more_than_four_characters2(all_cus_go['Subject'][5552]))

['zapytanie', 'amandine', 'perez']


In [32]:
# utworzenie nowej kolumny label i dodanie do niej słów, które nadają się na labelki

all_cus_go['label'] = [find_words_with_more_than_four_characters2(row) for row in all_cus_go['Subject']]

In [33]:
# sprawdzenie jak wygląda labels dla 10 przykładowych wierszy

all_cus_go['label'].sample(10)

57323                       [pomyłka]
43893    [etykieta, zwrotna, klienta]
7160                          [fehlt]
35173                    [reklamacja]
49481                      [etykieta]
13978                              []
58016                              []
28028                              []
3311                               []
38303            [mfrdyu, reklamacja]
Name: label, dtype: object

In [34]:
# Stworzenie listy ze wszystkimi wartościami z kolumny labels

label_list = []
[all_order(row, label_list) for row in all_cus_go['label']]
label_list = [item for sublist in label_list for item in sublist]

In [35]:
# Stworzenie dwóch list na podstawie listy wartości z kolumny['label'], jedna lista to unikatowe wartości, druga lista unikatowe wartości z informacją ile dana wartość występowała

uniq_vales, uniq_values_count = count_occurrences(label_list)

In [36]:
# Stworzenie obiektu spacy z polskim słownikiem

nlp = spacy.load("pl_core_news_md")

In [37]:
# Zmiana unikatowej listy na string

uniq_vales_sentence = ' '.join(uniq_vales)

In [38]:
# tokenizacja stringu unikatowych wartości z kolumny tags

doc = nlp(uniq_vales_sentence)

In [39]:
# lemantyzacja wcześniej stworzonych tokenów. Stworzenie listy i listy tupli. Lista tupli zawierające słowo i jej podstawę słowotwórczą, sama lista zawiera wszystkie podstwy słowotwórcze 

all_lem = []
all_words = []
for token in doc:
    all_words.append((token.text, token.lemma_))
    all_lem.append(token.lemma_)

In [40]:
# import pickle
# with open('all_lem.pkl', 'wb') as file:
#     pickle.dump(all_lem, file)

In [155]:
all_words

[('heiko', 'heiko'),
 ('hgtksxc', 'hgtksxc'),
 ('jednego', 'jeden'),
 ('zmzdya', 'zmzdya'),
 ('jxpqz', 'jxpqz'),
 ('numer', 'numer'),
 ('ckerstattung', 'ckerstattung'),
 ('bagnaschino', 'bagnaschino'),
 ('mrhess', 'mrhess'),
 ('katrincsonka', 'Katrincsonka'),
 ('ondatel', 'ondateli'),
 ('oczkuje', 'oczkować'),
 ('ymmdqlq', 'ymmdqlq'),
 ('nomination', 'nomination'),
 ('odbiorcy', 'odbiorca'),
 ('stefanie', 'stefanie'),
 ('bestway', 'bestway'),
 ('msrccu', 'msrccu'),
 ('wolfhard', 'wolfhard'),
 ('trpf', 'trpf'),
 ('wbtxo', 'wbtxo'),
 ('myyb', 'myyb'),
 ('kratki', 'kratka'),
 ('peter', 'peter'),
 ('awizacji', 'awizacja'),
 ('mqdxhg', 'Mqdxhg'),
 ('requiere', 'requiere'),
 ('wymaganych', 'wymagać'),
 ('gbbe', 'gbbe'),
 ('mfdpgu', 'mfdpgu'),
 ('mytqtt', 'mytqtt'),
 ('mkua', 'mkua'),
 ('osobisty', 'osobisty'),
 ('fedf', 'fedf'),
 ('khblv', 'khblv'),
 ('miskic', 'miskic'),
 ('mhha', 'mhha'),
 ('mqlpae', 'Mqlpae'),
 ('fehlender', 'fehlender'),
 ('powtierdzenie', 'powtierdzenie'),
 ('międzynaro

In [156]:
words_df = pd.DataFrame(all_words)

In [158]:
words_df.head()

,0,1
0,heiko,heiko
1,hgtksxc,hgtksxc
2,jednego,jeden
3,zmzdya,zmzdya
4,jxpqz,jxpqz


In [159]:
words_df.to_csv('all_words.csv', sep=',', encoding='cp1250',index=False)

In [ ]:
# Funkcja sprawdza czy peirwszy elemnt tupli jest kluczem do słownika, jeśli tak to dodaje 0 elemnt do listy i zwraca set, który zawiera unikatowe wartości dla danego klucza

def find_all_values_to_dictonary (key: str ,list_of_tuple: list) -> set:
    di = []
    for i in list_of_tuple:
        if i[1] == key:
            di.append(i[0])
    return set(di)


In [41]:
# Stworzenie słownika gdzie klucz to podstawa słowotwórcza a values jest listą słów dla tej podstawy występująca w kolumnie tags

lem_dict_0 = {}
for i in set(all_lem):
    lem_dict_0[i] = find_all_values_to_dictonary(i,all_words)

In [165]:
lem_dict_0['reklamajci']

{'reklamajca'}

In [42]:
# stworzenie słownika ze specjalnego obieku counter

uniq_values_count_dict = dict(uniq_values_count)

In [43]:
# Stworzenie słownika, gdzie klucz to podstawa słwowtwórcza, a values to suma wartości występowania słów zawiązanych z tą lemą

number_of_occurence = {}
for i in lem_dict_0:
    number_of_occurence_value = []
    for j in lem_dict_0[i]:
        number_of_occurence_value.append(uniq_values_count_dict[j])
    number_of_occurence[i] = sum(number_of_occurence_value)

In [44]:
# Posortowanie słownika zawierającego podstawę słowotwórczą oraz ilość wystąpnień słów z nim powiązanych

number_of_occurence_sorted = sorted(number_of_occurence.items(), key=lambda x:x[1], reverse=True)

In [45]:
df_number_of_occurence_sorted = pd.DataFrame(number_of_occurence_sorted, columns=['lema', 'ilość wystapień'])

In [46]:
df_number_of_occurence_sorted.head(5)

,lema,ilość wystapień
0,zwrot,6127
1,reklamacja,4539
2,dostawa,3243
3,niedostarczona,2812
4,anulować,2808


In [47]:
df_number_of_occurence_sorted.to_csv('number_of_occurence_sorted.csv', sep=',', encoding='cp1250',index=False)

In [166]:
labels_df = pd.read_csv('labels.csv', sep=';', header=None)
labels_df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,anulacja,anulować,anulacja,anulowac,anualcja,anulacj,anuowanie,anulownie,nieanulować,anulwoanie,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,reklamacja,reklamacja,reklamcja,reklaamcja,reklamajca,rekalamacjo,rekalmacja,reklmacja,rekalamcja,reklamac,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,wysyłka,wysyłce,wysyłka,wysłać,wysłek,wysyłke,wysyka,wysylce,wysyłką,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,refund,refund,refundacja,refunda,refunf,refung,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,przesyłka,przesyłka,przesylka,przesyłke,przesyłewka,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [167]:
labels_df = labels_df.T
labels_df.columns = labels_df.iloc[0]

In [168]:
labels_df

,anulacja,reklamacja,wysyłka,refund,przesyłka,dostawa,opóźnienie,uszkodzony,adres,ponowna,paczka,zapytanie,zwrot,pytanie,brak,niekompletne,faktura,inne
0,anulacja,reklamacja,wysyłka,refund,przesyłka,dostawa,opóźnienie,uszkodzony,adres,ponowna,paczka,zapytanie,zwrot,pytanie,brak,niekompletne,faktura,inne
1,anulować,reklamacja,wysyłce,refund,przesyłka,dostawa,opóźniić,uszkodzić,adres,ponowny,paczka,zapytanie,zwrot,pytanie,brak,niekompletny,faktura,inny
2,anulacja,reklamcja,wysyłka,refundacja,przesylka,niedostarczć,opóźnienie,uszkodzona,adresse,NaN,paczkomat,zapyatnie,zwrotny,pytanei,brakować,NaN,fattura,innen
3,anulowac,reklaamcja,wysłać,refunda,przesyłke,niedostarczić,opóźniyć,uszkodzeć,adresowy,NaN,paczk,zapyatanie,zwrtot,pytnaie,brakujący,NaN,fakura,NaN
4,anualcja,reklamajca,wysłek,refunf,przesyłewka,niedostarczono,opoznienie,uszkodozny,adresskl,NaN,paczkmać,zapyatenie,zwrto,pytnie,brakujyć,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,NaN,NaN,NaN,NaN,NaN,dostepność,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57,NaN,NaN,NaN,NaN,NaN,niedostarczpme,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58,NaN,NaN,NaN,NaN,NaN,niedostarczonea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,NaN,NaN,NaN,NaN,NaN,dostaroczić,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [169]:
lem_dict_1 = labels_df.to_dict(orient='list')

In [184]:
print(lem_dict_0['niedostarczić'])

KeyError: 'niedostarczić'

In [171]:
test1 = {'test1' : "['anulacja', 'anulować', 'anulacja', 'anulowac', 'anualcja', 'anulacj', 'anuowanie', 'anulownie', 'nieanulować', 'anulwoanie', 'anualacja', 'anuelowanie', 'anulownać', 'anuloewanie', 'nieanulowanie', 'anulownaie', 'anulaj', 'anuloanie', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]",}

In [172]:
test1
key_lvl1_test = []
key_lvl1_test.append(test1.values())
print([print(i) for i in key_lvl1_test])

dict_values(["['anulacja', 'anulować', 'anulacja', 'anulowac', 'anualcja', 'anulacj', 'anuowanie', 'anulownie', 'nieanulować', 'anulwoanie', 'anualacja', 'anuelowanie', 'anulownać', 'anuloewanie', 'nieanulowanie', 'anulownaie', 'anulaj', 'anuloanie', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]"])
[None]


In [185]:
from itertools import chain

def get_all_values (dict1: dict, dict0: dict) -> dict:
    final_dict = {}
    key_lvl1 = []
    key_lvl1.append(list(dict1.keys()))
    key_lvl1 = list(chain(*key_lvl1))
    for i in key_lvl1:
        temp_vals = []
        temp_keys = dict1[i]
        temp_keys = [x for x in temp_keys if x is not np.nan]
        for j in temp_keys:
            temp_vals.append(list(dict0[j]))
            print(temp_vals)
        temp_vals = set(item for sublist in temp_vals for item in sublist)
        final_dict[i] = temp_vals
        print(final_dict)
    return final_dict

In [186]:
get_all_values(lem_dict_1, lem_dict_0)

[['anulacji', 'anulację', 'anulacje', 'anulacja']]
[['anulacji', 'anulację', 'anulacje', 'anulacja'], ['anulować', 'anulowania', 'anulowanie', 'anulowani', 'anulowane', 'anulowana', 'anulowano']]
[['anulacji', 'anulację', 'anulacje', 'anulacja'], ['anulować', 'anulowania', 'anulowanie', 'anulowani', 'anulowane', 'anulowana', 'anulowano'], ['anulacji', 'anulację', 'anulacje', 'anulacja']]
[['anulacji', 'anulację', 'anulacje', 'anulacja'], ['anulować', 'anulowania', 'anulowanie', 'anulowani', 'anulowane', 'anulowana', 'anulowano'], ['anulacji', 'anulację', 'anulacje', 'anulacja'], ['anulowac']]
[['anulacji', 'anulację', 'anulacje', 'anulacja'], ['anulować', 'anulowania', 'anulowanie', 'anulowani', 'anulowane', 'anulowana', 'anulowano'], ['anulacji', 'anulację', 'anulacje', 'anulacja'], ['anulowac'], ['anualcji', 'anualcja']]
[['anulacji', 'anulację', 'anulacje', 'anulacja'], ['anulować', 'anulowania', 'anulowanie', 'anulowani', 'anulowane', 'anulowana', 'anulowano'], ['anulacji', 'anulac

KeyError: 'reklamajca'

In [148]:
type(lem_dict_0['anulownać'])

KeyError: 'anulownać'

In [123]:
lem_dict_0_df = pd.DataFrame(lem_dict_0)

ValueError: All arrays must be of the same length